## Reduction: the sum of the elements of an array

In [4]:
import sys
#parámetro desde la línea de comandos
value = 5*10**7
if len(sys.argv) > 1: #Usa el último argumento de la línea si es un entero
    last = sys.argv[-1]
    if last.isdigit():
        value = int(last)

print(f"El valor de 'value' es: {value}")

El valor de 'value' es: 50000000


In [2]:
import numpy as np

def reduc_operation(A):
    """Compute the sum of the elements of Array A."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# Secuencial


X = np.random.rand(value)

# Para imprimir los primeros valores del array

#print(X[0:12])

# Utilizando las operaciones mágicas de ipython

tiempo = %timeit -r 2 -o -q reduc_operation(X)

print("Time taken by reduction operation using a function:", tiempo)

print(f"And the result of the sum of numbers in the range [0, value) is: {reduc_operation(X)}\n")


# Utilizando numpy.sum()

tiempo = %timeit -r 2 -o -q np.sum(X)

print("Time taken by reduction operation using numpy.sum():", tiempo)

print("Now, the result using numpy.sum():", np.sum(X),"\n ")


Time taken by reduction operation using a function: 5.53 s ± 15.7 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result of the sum of numbers in the range [0, value) is: 25001971.22988875

Time taken by reduction operation using numpy.sum(): 19 ms ± 4.19 μs per loop (mean ± std. dev. of 2 runs, 100 loops each)
Now, the result using numpy.sum(): 25001971.22989084 
 


Apartado a) Usando el paquete multiprocessing

In [3]:
import time
from multiprocessing import Pool


def reduc_operation(A):
    """Compute the sum of the elements of Array A."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s
    
def reduc_operation_parallel(X, ncores):
    parts= np.array_split(X,ncores)
    with Pool(processes=ncores) as p:
        partial= p.map(reduc_operation, parts)
    return sum(partial)
        

X = np.random.rand(value)
ncores=2

for ncores in [1, 2, 4]:
    start = time.time()
    total = reduc_operation_parallel(X, ncores)
    end = time.time()
    print(f"Resultado usando {ncores} procesos: {total}, Tiempo: {end - start:.4f} segundos")


Resultado usando 1 procesos: 24998410.371808775, Tiempo: 5.6585 segundos
Resultado usando 2 procesos: 24998410.37180812, Tiempo: 3.0249 segundos
Resultado usando 4 procesos: 24998410.37180774, Tiempo: 1.6992 segundos


Apartado b) Usando el decorador @njit del paquete Numba

In [1]:
from numba import njit

@njit
def reduc_operation(A):
    """Compute the sum of the elements of Array A."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

X = np.random.rand(value)

tiempo = %timeit -r 2 -o -q reduc_operation(X)

print("Time taken by reduction operation using a function:", tiempo)

print(f"And the result of the sum of numbers in the range [0, value) is: {reduc_operation(X)}\n")


tiempo = %timeit -r 2 -o -q np.sum(X)

print("Time taken by reduction operation using numpy.sum():", tiempo)

print("Now, the result using numpy.sum():", np.sum(X),"\n ")


Time taken by reduction operation using a function: 49.4 ms ± 4.05 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result of the sum of numbers in the range [0, value) is: 25001275.335583303

Time taken by reduction operation using numpy.sum(): 18.9 ms ± 28.5 μs per loop (mean ± std. dev. of 2 runs, 100 loops each)
Now, the result using numpy.sum(): 25001275.335583158 
 


Apartado b) Usando el decorador @njit(parallel=T) y prange del paquete Numba

In [2]:
from numba import prange

@njit (parallel=True)
def reduc_operation(A):
    """Compute the sum of the elements of Array A."""
    s = 0
    for i in prange(A.size):
        s += A[i]
    return s

# Secuencial

X = np.random.rand(value)

# Para imprimir los primeros valores del array

#print(X[0:12])

# Utilizando las operaciones mágicas de ipython

tiempo = %timeit -r 2 -o -q reduc_operation(X)

print("Time taken by reduction operation using a function:", tiempo)

print(f"And the result of the sum of numbers in the range [0, value) is: {reduc_operation(X)}\n")


# Utilizando numpy.sum()

tiempo = %timeit -r 2 -o -q np.sum(X)

print("Time taken by reduction operation using numpy.sum():", tiempo)

print("Now, the result using numpy.sum():", np.sum(X),"\n ")


Time taken by reduction operation using a function: 11.6 ms ± 53.8 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result of the sum of numbers in the range [0, value) is: 24998129.08773788

Time taken by reduction operation using numpy.sum(): 18.9 ms ± 13.6 μs per loop (mean ± std. dev. of 2 runs, 100 loops each)
Now, the result using numpy.sum(): 24998129.08773774 
 


EJECUCIÓN CON 1 NÚCLEO

El valor de 'number' es 10**8

1. Código original de Python: función= 17.3 s ± 53.1 ms per loop /np.sum= 64.1 ms ± 35.4 μs per loop
2. Multiprocessing con Pool: 1 proceso= 21.0642 segundos/ 2 procesos= 11.3521 segundos/ 4 procesos= 6.2975 segundos
4. Numba secuencial: función= 52.5 ms ± 3.65 ms per loop / np.sum= 64.5 ms ± 83.2 μs per loop
5. Numba paralelo + prange: función= 34.7 ms ± 573 μs per loop /np.sum= 94.6 ms ± 8.43 ms per loop

El valor de 'number' es: 10**9

1. Código original de Python: función= 3min 14s ± 159 ms per loop /np.sum= 641 ms ± 373 μs per loop
2. Multiprocessing con Pool: 1 proceso= 230.6096 segundos/ 2 procesos= 122.6295 segundos/ 4 procesos= 66.7304 segundos
3. Numba secuencial: función= 303 ms ± 48 ms per loop / np.sum= 695 ms ± 8.84 ms per loop
4. Numba paralelo + prange: función=  352 ms ± 13.1 ms per loop /np.sum= 649 ms ± 305 μs per loop


EJECUCIÓN CON 2 NÚCLEOS

El valor de 'number' es 10**8

1. Código original de Python: función= 17.3 s ± 45.1 ms per loop /np.sum= 64.1 ms ± 28.7 μs per loop
2. Multiprocessing con Pool: 1 proceso= 21.1761 segundos/ 2 procesos= 11.1321 segundos/ 4 procesos= 6.2656 segundos
3. Numba secuencial: función= 117 ms ± 116 μs per loop / np.sum= 77.1 ms ± 51.9 μs per loop
4. Numba paralelo + prange: función= 69 ms ± 2.98 ms per loop /np.sum= 78.5 ms ± 1.37 ms per loop

El valor de 'number' es: 10**9

1. Código original de Python: función= 2min 52s ± 762 ms per loop /np.sum= 693 ms ± 240 μs per loop
2. Multiprocessing con Pool: 1 proceso= 211.7212 segundos/ 2 procesos= 114.2238 segundos/ 4 procesos= 63.4239 segundos
3. Numba secuencial: función= 1.16 s ± 336 ns per loop / np.sum= 693 ms ± 11.6 ms per loop
4. Numba paralelo + prange: función= 342 ms ± 21.9 ms per loop /np.sum= 749 ms ± 1.31 ms per loop


EJECUCIÓN CON 4 NÚCLEOS

El valor de 'number' es 10**8

1. Código original de Python: función= 17.3 s ± 41.9 ms per loop /np.sum= 64.1 ms ± 5.49 μs per loop
2. Multiprocessing con Pool: 1 proceso= 21.4035 segundos/ 2 procesos= 11.5190 segundos/ 4 procesos= 6.4039 segundos
3. Numba secuencial: función= 116 ms ± 75.8 μs per loop / np.sum= 69.9 ms ± 821 ns per loop
4. Numba paralelo + prange: función= 27.3 ms ± 2.7 ms per loop /np.sum= 76.8 ms ± 7.37 ms per loop

El valor de 'number' es: 10**9

1. Código original de Python: función= 2min 52s ± 451 ms per loop /np.sum= 693 ms ± 351 μs per loop
2. Multiprocessing con Pool: 1 proceso= 211.1228 segundos/ 2 procesos= 113.1237 segundos/ 4 procesos= 62.7768 segundos
3. Numba secuencial: función= 1.16 s ± 97.9 μs per loop / np.sum= 690 ms ± 8.9 ms per loop
4. Numba paralelo + prange: función= 345 ms ± 9.16 ms per loop /np.sum= 718 ms ± 27.9 ms per loop


EJECUCIÓN CON 8 NÚCLEOS

El valor de 'number' es 10**8

1. Código original de Python: función= 17.3 s ± 45.2 ms per loop /np.sum= 64 ms ± 4.19 μs per loop
2. Multiprocessing con Pool: 1 proceso= 20.9817 segundos/ 2 procesos= 11.1964 segundos/ 4 procesos= 6.2179 segundos
3. Numba secuencial: función= 117 ms ± 68.4 μs per loop / np.sum= 77.1 ms ± 21.5 μs per loop
4. Numba paralelo + prange: función= 61.5 ms ± 4.99 ms per loop /np.sum= 77.4 ms ± 1.49 ms per loop

El valor de 'number' es: 10**9

1. Código original de Python: función= 2min 52s ± 436 ms per loop /np.sum= 652 ms ± 322 μs per loop
2. Multiprocessing con Pool: 1 proceso= 230.3097 segundos/ 2 procesos= 124.4115 segundos/ 4 procesos= 67.9039 segundos
3. Numba secuencial: función= 1.16 s ± 4.35 ms per loop / np.sum= 642 ms ± 17.8 μs per loop
4. Numba paralelo + prange: función= 304 ms ± 8.91 ms per loop /np.sum= 717 ms ± 7.77 μs per loop


## **CONCLUSIÓN DE LOS 3 ENFOQUES ESTUDIADOS:**

En este notebook se comparan el tiempo de ejecución de una operación de reducción usando tres enfoques distintos, usando tamaños de problemas grandes y variando el número de núcleos asignados en SLURM. 
Según los resultados obtenidos se puede ver como el código original de Python es el más lento debido al overhead del intérprete y al uso de bucles, y por ello tampoco mejora con el aumento de número de núcleos. Por otro lado, el uso de multiprocessing sí permite una reducción del tiempo ya que el trabajo se reparte entre varios procesos. Sin embargo debido a su coste de gestión, ninguno de los tiempos obtenidos mejora con respecto al generado por numpy.sum() y porque este está implementada en código C optimizado.
Como muestran los resultados, Numba se trata de la opción mas eficiente para este tipo de problemas sobre grandes volúmenes de datos. Los tiempos se reducen debido a que el código se compila a máquina e incluso numba con prange es más rápida que numpy.sum() en varios casos ya que combina código compilado con paralelismo con varios hilos. 
Además, en cuánto al número de núcleos puede observarse que usar más núcleos no siempre hace que el programa sea más rápido, porque el tiempo de ejecución está limitado por el acceso a la memoria y el coste extra de gestionar el paralelismo.

